# Web scraping

In [ ]:
from bs4 import BeautifulSoup
import requests
import re

In [ ]:
FIRST_VOTING = 67010 # first voting of parliament
LAST_VOTING = 73895 # last voting id of parliament

COLUMNS_MOP = ['Jmeno', 'Strana', 'HlasovaniId', 'Hlas', 'Datum'] # Memeber of parliament record description
COLUMNS_VOTING = ['Id', 'Title', 'Date', 'TotalResult'] # Voting record description
MONTHS = {'ledna' : 1, 'února' : 2, 'března' : 3, 'dubna' : 4, 'května' : 5, 'června' : 6, 'července' : 7,
         'srpna' : 8, 'září' : 9, 'října' : 10, 'listopadu' : 11, 'prosince' : 12} # Helper for parsing the dates
PARTIES = {'ANO', 'ODS', 'Piráti', 'SPD', 'ČSSD', 'KSČM', 'KDU-ČSL', 'TOP09', 'STAN', 'Nezařaz'}

In [ ]:
# Helper function for date parsing
def parse_date(date):
    for m in months:
        m = m.lower()
        if date.find(m) > 0:
            date = date.replace(m, str(months[m]))
            return parser.parse(date)
    return None

def get_voting_name(title):
    for i in range(5, 1, -1):
        try:
            return title.contents[i]
        except:
            continue;
            #print('Title is misssing, trying again {0}'.format(i))   

def get_date(title):
    try:
        date = parse_date(str(title.contents[0]).replace('\xa0',' ')[2:])
        return date
    except:
        return None

def parse_party(party):
    idx = str(party.contents[0]).find(' ')
    return str(party.contents[0])[6:idx]

        
# Parse a voting record
def parse_voting(voting_n, soup):
    print('parsing')
    # Init df
    ret = pd.DataFrame(columns=COLUMNS_VOTING)
    ret = ret.set_index('Id')
    # Get basic web section 
    title = soup.find('h1', {'class': 'page-title-x'})
    result = soup.find('h2', {'class': 'section-title center'}).find('span').contents[0]
    
    date = get_date(title)
    voting_name = get_voting_name(title)
          
    ret = ret.append({'Id' : voting_n,
    'Title' : voting_name,
    'Date' : date,
    'TotalResult' : 'accepted' if str(result).find('PŘIJAT') > 0 else 'rejected'},ignore_index=True)
    return ret


def parse_result(result, party, voting_n, date, df):
    mop_from_party = result.find_all('li')
    ret = list()
    for mop in mop_from_party:
        name = str(mop.find('a').contents[0]).replace('\xa0',' ')
        vote_res = mop.find('span')['class'][1]
        ret.append(pd.Series([name, party, voting_n,vote_res, date], index=df.columns))
    return ret

def parse_results(results, parties, voting_n, date):
    party_idx = 0
    ret = pd.DataFrame(columns=COLUMNS_MOP)
    parties_parsed = list() 
    
    for party in parties:
        parties_parsed.append(parse_party(party))
        
    for r in results:
        res = parse_result(r, parties_parsed[party_idx], voting_n, date, ret)
        ret = ret.append(res, ignore_index=True)
        party_idx += 1
        
    return ret
    
    
def get_results_from_voting(voting_n):
    url = 'https://www.psp.cz/sqw/hlasy.sqw?g={0}&l=cz'.format(voting_n)
    html_content = requests.get(url)
    soup = BeautifulSoup(html_content.text, 'html.parser')
    results = soup.find_all('ul', {"class" : "results"})
    strany = soup.find('div', {'id' : 'main-content'}).find_all('h2', {'class' : 'section-title'}, recursive=False)
    voting = parse_voting(voting_n, soup)
    return parse_results(results, strany, voting_n, voting['Date'][0]), voting

In [ ]:
results, voting = get_results_from_voting(67018)
df_r = results
df_v = voting
for i in range(67019, 73895):
    return
    if i % 200 == 0: # Every 200 votings, make a backup
        curr_r = None
        curr_v = None
        try:
            curr_r = pd.read_csv('res.csv', encoding='utf-8-sig')
            curr_v = pd.read_csv('res_voting.csv',encoding='utf-8-sig')
        except:
            print('Creating sotrage')
        if curr_v is not None:       
            df_r = pd.concat([curr_r, df_r])
            df_v = pd.concat([curr_v, df_v])
            
        df_r.to_csv('res.csv', index=False, encoding='utf-8-sig')
        df_v.to_csv('res_voting.csv', index=False, encoding='utf-8-sig')
        
        df_r = df_r.iloc[0:0]
        df_v = df_v.iloc[0:0]
    try:
        r, v = get_results_from_voting(i)
    except:
        print('Voting {0} doesnt exist'.format(i))
    df_r = pd.concat([r, df_r])
    df_v = pd.concat([v, df_v])
    time.sleep(1)
    if i % 10 == 0: # Visualize progress
        print(i)